<span style="font-family: 'Times New Roman'; font-size: 24px; color: white; font-weight: bold;">Preparation of the Data Test</span>

<span style="font-family: 'Times New Roman'; font-size: 22px; color: white; font-weight: bold;">0. Import libraries and load data</span>

In [1]:
import pandas as pd
from openpyxl import load_workbook
import os
import numpy as np

In [4]:
parent_dir = os.path.abspath('..')
data_path = os.path.join(parent_dir, 'datasets')
files = os.listdir(data_path)
files.remove('URL descripcion reto.txt')
files.remove('leeme.txt')
dfs = {}

def open_file(file_name):
    """
    Function to load all sheets from an Excel file into a global dictionary as separate DataFrames.

    Args:
        file_name (str): The name of the Excel file to be loaded (must be located in the directory defined by 'data_path').

    Functionality:
    - Constructs the full file path by combining the base data directory ('data_path') and the given file name.
    - Reads all sheets from the specified Excel file into a dictionary, where each sheet is a key-value pair 
      (key: sheet name, value: corresponding DataFrame).
    - Assigns a unique name to each DataFrame by combining the file name (without its '.xlsx' extension) and the sheet name.
    - Stores each DataFrame in a global dictionary 'dfs', allowing for easy access using the unique names as keys.

    Note:
    - This function relies on the global variables 'data_path' (directory path) and 'dfs' (dictionary to store DataFrames).
    """

    # Construct the full file path using the base directory and the file name.
    file_path = os.path.join(data_path, file_name)

    # Read all sheets from the Excel file into a dictionary.
    # Each key represents a sheet name, and its value is the corresponding DataFrame.
    all_sheets = pd.read_excel(file_path, sheet_name=None)

    # Iterate over all sheets in the dictionary.
    for sheet_name, data in all_sheets.items():
        # Generate a unique name for the DataFrame by combining the file name (excluding '.xlsx') with the sheet name.
        df_name = f"{file_name.replace('.xlsx', '')}_{sheet_name}"
        
        # Store the DataFrame in the global dictionary 'dfs' using the unique name as the key.
        dfs[df_name] = data

# Open all files and read all sheets
for file in files:
    if file.endswith('.xlsx'):  # Ensure only Excel files are processed
        open_file(file)
    if file.endswith('.XLSX'):  # Handle case-sensitive file extensions
        open_file(file)

In [5]:
print(dfs.keys())

dict_keys(['Biorreactor 14618_WData', 'Biorreactor 14618_Datos', 'Biorreactor 13169_WData', 'Biorreactor 13169_Datos', 'Biorreactor 14614_Datos', 'Biorreactor 14615_WData', 'Biorreactor 14615_Datos', 'Horas inicio fin centrífugas_Hoja1', 'Centrífuga 17825_WData', 'Centrífuga 17825_Datos', 'Biorreactor 13172_WData', 'Biorreactor 13172_Datos', 'Cinéticos IPC_Inóculos', 'Cinéticos IPC_Cultivos finales', 'Cinéticos IPC_Centrifugación', 'Centrífuga 12912_WData', 'Centrífuga 12912_Datos', 'Fases producción_test v02_Cultivo final', 'Fases producción v03 Test_Cultivo final', 'Temperaturas y humedades_WData', 'Temperaturas y humedades_Datos', 'Fases producción v02_Preinóculo', 'Fases producción v02_Inóculo', 'Fases producción v02_Cultivo final', 'Biorreactor 13171_WData', 'Biorreactor 13171_Datos', 'OF 123456 v03_Sheet1', 'Biorreactor 14616_WData', 'Biorreactor 14616_Datos', 'Centrífuga 14246_WData', 'Centrífuga 14246_Datos', 'Biorreactor 14617_WData', 'Biorreactor 14617_Datos', 'Movimientos co

In [6]:
# Access to the Dataframes
df_preinoculo = dfs['Fases producción v03_Preinóculo']
df_preinoculo.columns = df_preinoculo.iloc[0]
df_preinoculo = df_preinoculo.drop(0)
df_inoculo = dfs['Fases producción v03_Inóculo']
df_cultivo_final = dfs['Fases producción v03 Test_Cultivo final']

In [7]:
# merge dataframes by 'LOTE'
df_of_123456 = dfs['OF 123456 v03_Sheet1']
df_of_123456 = df_of_123456.rename(columns={df_of_123456.columns[3]: 'LOTE'})
df_cineticos = dfs['Cinéticos IPC_Inóculos']
df_cineticos = df_cineticos.rename(columns={df_cineticos.columns[0]: 'LOTE'})
df_cineticos = df_cineticos[['LOTE', 'Turbidez', 'Viabilidad']].groupby('LOTE',as_index=False).mean()
df_of_123456['LOTE'] = df_of_123456['LOTE'].astype(str).str.replace('/', '').astype(int)
# Unir los tres DataFrames por la columna 'LOTE'
merged_df = df_preinoculo.merge(df_inoculo, on='LOTE', suffixes=('_preinoculo', '_inoculo'))
merged_df = merged_df.merge(df_of_123456[['LOTE', 'Cantidad entregada']], on='LOTE', suffixes=('', '_OF_123456'))

merged_df = merged_df.merge(df_cineticos, on='LOTE', suffixes=('', 'cineticos'))
# Unir los tres DataFrames por la columna 'LOTE'
merged_df = merged_df.merge(df_inoculo, on='LOTE', suffixes=('', '_inoculo'))
merged_df = merged_df.merge(df_cultivo_final, on='LOTE', how='right', suffixes=('', '_cultivo_final'))

/var/folders/89/t9vgkdv52f57k0g7jq3p9c4h0000gn/T/ipykernel_2241/1819826768.py:15: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  merged_df = merged_df.merge(df_cultivo_final, on='LOTE', how='right', suffixes=('', '_cultivo_final'))


<span style="font-family: 'Times New Roman'; font-size: 22px; color: white; font-weight: bold;">1. Data TEST Preparation</span>

In [8]:
merged_df.shape

(55, 43)

In [9]:
merged_df['ID Centrífuga'] = merged_df['ID Centrífuga'].astype(int)
merged_df['ID Bioreactor'] = merged_df['ID Bioreactor'].astype(int)
mode_value = merged_df['ID bioreactor'].mode().iloc[0]  # Get the first mode
merged_df['ID bioreactor'].fillna(mode_value, inplace=True)
merged_df['ID bioreactor'].replace([float('inf'), -float('inf')], mode_value, inplace=True)
merged_df['ID bioreactor'] = merged_df['ID bioreactor'].astype(int)

In [10]:
def load_centrifuga_data(id_centrifuga):
    """
    Loads centrifuge data for a specific centrifuge ID from the global 'dfs' dictionary.

    Args:
        id_centrifuga (str or int): The ID of the centrifuge to load data for.

    Returns:
        pd.DataFrame: The DataFrame corresponding to the centrifuge data if found.
                      If the data is not available, returns an empty DataFrame.
    """
    # Construct the file name for the centrifuge data based on the ID.
    file_name = f'Centrífuga {id_centrifuga}_Datos'
    
    try:
        # Attempt to load the centrifuge DataFrame from the 'dfs' dictionary.
        centrifuga_df = dfs[file_name]
        return centrifuga_df
    except KeyError:
        # If the file does not exist, return an empty DataFrame as a fallback.
        return pd.DataFrame()  # Alternatively, use 'return np.nan' if preferred.

In [11]:
def calculate_mean_for_lote(lote, fecha_inicio, fecha_fin, id_centrifuga):
    """
    Calculates the mean value of a specific parameter for a centrifuge within a given date range.

    Args:
        lote (str): The batch or lot identifier (not used directly in this function but may be relevant for context).
        fecha_inicio (str): The start date for filtering data (in a format compatible with pandas.to_datetime).
        fecha_fin (str): The end date for filtering data (in a format compatible with pandas.to_datetime).
        id_centrifuga (str or int): The centrifuge ID to load data for.

    Returns:
        float: The mean value of the specified parameter for the centrifuge within the date range.
               Returns NaN if no data is available.
    """
    # Load the data for the specified centrifuge.
    centrifuga_df = load_centrifuga_data(id_centrifuga)

    # Check if the DataFrame is empty. If no data is available, return NaN.
    if centrifuga_df.empty:
        return np.nan

    # Filter the data based on the date range.
    mask = (
        (pd.to_datetime(centrifuga_df['DateTime']) >= pd.to_datetime(fecha_inicio)) &
        (pd.to_datetime(centrifuga_df['DateTime']) <= pd.to_datetime(fecha_fin))
    )
    filtered_data = centrifuga_df[mask]

    # Calculate and return the mean value of the specified parameter column.
    return filtered_data[f'{id_centrifuga}_D01916047.PV'].mean()

In [12]:
def calculate_mean_for_lote_temp(lote, fecha_inicio, fecha_fin):
    """
    Calculates the mean values of specific temperature and humidity parameters within a given date range.

    Args:
        lote (str): The batch or lot identifier (not directly used in this function but may be relevant for context).
        fecha_inicio (str): The start date for filtering data (in a format compatible with pandas.to_datetime).
        fecha_fin (str): The end date for filtering data (in a format compatible with pandas.to_datetime).

    Returns:
        pd.Series: A Series containing the mean values for the specified parameters.
                   Returns NaN if no data is available.
    """
    # Load the DataFrame containing temperature and humidity data.
    temp_df = dfs['Temperaturas y humedades_Datos']

    # Check if the DataFrame is empty. If no data is available, return NaN.
    if temp_df.empty:
        return np.nan

    # Filter the data based on the date range.
    mask = (
        (pd.to_datetime(temp_df['DateTime']) >= pd.to_datetime(fecha_inicio)) &
        (pd.to_datetime(temp_df['DateTime']) <= pd.to_datetime(fecha_fin))
    )
    filtered_data = temp_df[mask]

    # Calculate and return the mean values for the specified parameters.
    return filtered_data[['07781_TI1501.PV', '07781_MI1501.PV']].mean()

In [13]:
def load_bioreactor_data(id_bioreactor):
    """
    Loads bioreactor data for a specific bioreactor ID from the global 'dfs' dictionary.

    Args:
        id_bioreactor (str or int): The ID of the bioreactor to load data for.

    Returns:
        pd.DataFrame: The DataFrame containing bioreactor data if found.
                      If the data is not available, returns an empty DataFrame.
    """
    # Construct the file name for the bioreactor data based on the given ID.
    file_name = f'Biorreactor {id_bioreactor}_Datos'
    
    try:
        # Attempt to retrieve the DataFrame from the global 'dfs' dictionary.
        bioreactor_df = dfs[file_name]
        return bioreactor_df
    except KeyError:
        # If the file is not found in the dictionary, return an empty DataFrame as a fallback.
        return pd.DataFrame()  # Alternatively, use 'return np.nan' if preferred.

In [14]:
def calculate_mean_for_lote2(lote, fecha_inicio, fecha_fin, id_bioreactor):
    """
    Calculates the mean values for temperature, pH, and dissolved oxygen (DO) 
    for a specific lot and bioreactor within a given date range.

    Args:
        lote (str): The batch or lot identifier (not directly used in this function but may be relevant for context).
        fecha_inicio (str): The start date for filtering data (in a format compatible with pandas.to_datetime).
        fecha_fin (str): The end date for filtering data (in a format compatible with pandas.to_datetime).
        id_bioreactor (str or int): The bioreactor ID to load data for.

    Returns:
        tuple: A tuple containing the mean values for:
            - Temperature (`mean_temp`)
            - pH (`mean_ph`)
            - Dissolved Oxygen (`mean_do`)
            Returns NaN for each value if no data is available.
    """
    # Load data for the specified bioreactor.
    bioreactor_df = load_bioreactor_data(id_bioreactor)

    # If the DataFrame is empty, return NaN for all calculated means.
    if bioreactor_df.empty:
        return np.nan, np.nan, np.nan

    # Filter the data based on the specified date range.
    mask = (
        (pd.to_datetime(bioreactor_df['DateTime']) >= pd.to_datetime(fecha_inicio)) &
        (pd.to_datetime(bioreactor_df['DateTime']) <= pd.to_datetime(fecha_fin))
    )
    filtered_data = bioreactor_df[mask]

    # Calculate mean values for the specified columns. Return NaN if no data is available after filtering.
    mean_temp = filtered_data[f'{id_bioreactor}_FERM0101.Temperatura_PV'].mean() if not filtered_data.empty else np.nan
    mean_ph = filtered_data[f'{id_bioreactor}_FERM0101.Single_Use_pH_PV'].mean() if not filtered_data.empty else np.nan
    mean_do = filtered_data[f'{id_bioreactor}_FERM0101.Single_Use_DO_PV'].mean() if not filtered_data.empty else np.nan

    # Return the mean values as a tuple.
    return mean_temp, mean_ph, mean_do

In [15]:
# Apply the function to each row of merged_df and store the results in new columns
merged_df[['media_temp_bioreactor', 'media_ph_biorreactor', 'media_PO_biorreactor']] = merged_df.apply(
    lambda row: calculate_mean_for_lote2(row['LOTE'], row['Fecha/hora inicio_inoculo'], row['Fecha/hora fin_inoculo'], row['ID bioreactor']),
    axis=1,
    result_type='expand'  # This allows unpacking into multiple columns
)

In [16]:
merged_df = merged_df.fillna("NA")
output_file_path = 'dataset_test_fase2.csv'  # Cambia esto a la ruta donde quieras guardar el archivo
merged_df.to_csv(output_file_path, index=False)

In [17]:
merged_df

,LOTE,Fecha/hora inicio_preinoculo,Fecha/hora fin_preinoculo,línea 1,línea 2,línea 3,línea 1,línea 2,línea 3,línea 1,...,Turbidez fin cultivo,Viabilidad final cultivo_cultivo_final,ID Centrífuga,Centrifugación 1 turbidez,Centrifugación 2 turbidez,Producto 1,Producto 2,media_temp_bioreactor,media_ph_biorreactor,media_PO_biorreactor
0,24054.0,2024-04-14 00:30:00,2024-04-15 07:17:00,5.504,5.376,5.376,27.12,25.68,29.52,1,...,85.60,184800000,14246,27.84,23.96,NA,NA,29.523502,5.889634,23.939733
1,24055.0,2024-04-11 00:30:00,2024-04-12 07:14:00,5.368,5.416,5.464,26.16,25.52,25.04,1,...,73.68,175200000,12912,30.96,23.16,NA,NA,29.478246,5.909516,24.440044
2,24056.0,2024-04-11 00:30:00,2024-04-12 07:14:00,5.368,5.416,5.464,26.16,25.52,25.04,1,...,82.40,168000000,14246,29.52,28.88,NA,NA,29.478246,5.909516,24.440044
3,24057.0,2024-04-14 00:30:00,2024-04-15 07:17:00,5.504,5.376,5.376,27.12,25.68,29.52,1,...,78.96,180800000,12912,31.04,25.32,NA,NA,29.523502,5.889634,23.939733
4,24058.0,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,82.40,144800000,12912,26.08,20.36,NA,NA,NA,NA,NA
5,24059.0,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,78.40,166400000,14246,26.88,23.64,NA,NA,NA,NA,NA
6,24060.0,2024-04-18 00:30:00,2024-04-19 07:46:00,5.384,5.344,5.448,25.44,26,24.72,1,...,76.32,161600000,14246,32.56,17.92,NA,NA,29.480435,5.881805,24.754048
7,24061.0,2024-04-18 00:30:00,2024-04-19 07:46:00,5.384,5.344,5.448,25.44,26,24.72,1,...,68.96,148800000,12912,27.76,14.72,NA,NA,29.480435,5.881805,24.754048
8,24062.0,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,80.00,164800000,12912,27.52,24.64,NA,NA,NA,NA,NA
9,24063.0,2024-04-21 00:30:00,2024-04-22 07:19:00,5.36,5.376,NA,25.84,27.84,NA,1,...,68.72,180800000,12912,25.92,18.28,NA,NA,29.46004,5.866393,24.954162
